In [145]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re

In [146]:
data = pd.read_csv('./despesas_classificadas.csv', sep=',')

In [147]:
# palavra = "ALLIANZ"

# data['Categoria'] = data['DESCRIÇÃO'].apply(lambda x: 'Seguro' if palavra.lower() in x.lower() else data.loc[data['DESCRIÇÃO'] == x, 'Categoria'].values[0])

# data.to_csv('./despesas_classificadas.csv', sep=',', index=False)


In [148]:
def tratar_dados(data):

    data = str(data).lower()  
    data = re.sub(r"[.,-/]", " ", data)  
    data = re.sub(r"[^a-z\s]", "", data) 
    data = re.sub(r"\s+", " ", data)  
    data = data.strip()  
    return data

In [149]:
data['DESCRIÇÃO'] = data['DESCRIÇÃO'].apply(tratar_dados)

In [150]:
output_path = './despesas_classificadas_tratadas.csv'
data.to_csv(output_path, index=False)

In [151]:
df = pd.read_csv(output_path, sep=',')

In [152]:
df['Categoria_Codificada'] = LabelEncoder().fit_transform(df['Categoria'])

In [153]:
df.head(20)

,DATA,DESCRIÇÃO,DOCUMENTO,CRÉDITO,DÉBITO,SALDO,Categoria,Categoria_Codificada
0,1/4/2022,pg pinternet white martins,10.002.006,NaN,"-R$ 30,90","R$ 16.284,18",Despesas Diversas,2
1,1/4/2022,pg pinternet med curitiba,10.001.995,NaN,"-R$ 100,00","R$ 16.184,18",Saúde,8
2,1/4/2022,pg pinternet springer carrier,10.002.010,NaN,"-R$ 358,91","R$ 15.825,27",Equipamentos,5
3,1/4/2022,credito ted pagseguro internet ltda,33.834.846,"R$ 846,95",NaN,"R$ 16.672,22",Serviço Prestado,10
4,1/7/2022,pg pinternet aluguel,10.005.388,NaN,"-R$ 3.046,25","R$ 13.625,97",Aluguel,0
5,1/10/2022,prest emprest,8,NaN,"-R$ 1.418,47","R$ 12.207,50",Empréstimos,4
6,1/10/2022,db allianz seguro spin de,84624000166,NaN,"-R$ 144,05","R$ 12.063,45",Seguro,9
7,1/11/2022,credito ted pagseguro internet ltda,33.655.713,"R$ 1.449,70",NaN,"R$ 13.513,15",Serviço Prestado,10
8,1/17/2022,db cotas,19.911,NaN,"-R$ 59,50","R$ 13.453,65",Despesas Diversas,2
9,1/17/2022,credito ted pagseguro internet ltda,16,"R$ 1.517,30",NaN,"R$ 14.970,95",Serviço Prestado,10


In [154]:
X_train, X_test, y_train, y_test = train_test_split(df['DESCRIÇÃO'], df['Categoria_Codificada'], test_size=0.2, random_state=42)

In [155]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [156]:
rf_model = RandomForestClassifier(random_state=42, n_estimators=500)
rf_model.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=500, random_state=42)

### Aplicando o modelo direto (sem balanceamento)

In [157]:
y_pred = rf_model.predict(X_test_tfidf)

In [158]:
present_labels = sorted(set(y_test))  # Identificar as classes presentes no conjunto de teste
report = classification_report(y_test, y_pred, labels=present_labels)

In [159]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy * 100:.2f}%\n")
print("Relatório de Classificação:\n", report)

Acurácia: 97.47%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.93      1.00      0.97        14
           2       0.94      0.98      0.96        48
           3       1.00      1.00      1.00         3
           4       1.00      0.83      0.91         6
           5       1.00      1.00      1.00        15
           6       1.00      0.86      0.92         7
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         8
          10       1.00      1.00      1.00        42
          12       1.00      0.83      0.91         6

    accuracy                           0.97       158
   macro avg       0.99      0.96      0.97       158
weighted avg       0.98      0.97      0.97       158



### Validação do modelo com StratifiedKFold

In [160]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

X = df['DESCRIÇÃO']
y = df['Categoria_Codificada']


X_tfidf = vectorizer.transform(X)


cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(rf_model, X_tfidf, y, cv=cv, scoring='accuracy')

d:\Repositories\AVD\venv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [161]:
results = pd.DataFrame({
    'Acurácia': scores,
    'Fold': range(1, len(scores) + 1)
})

results.head(10) 

,Acurácia,Fold
0,1.000000,1
1,0.974684,2
2,0.987342,3
3,0.987342,4
4,0.987342,5
5,0.974684,6
6,1.000000,7
7,1.000000,8
8,0.987179,9
9,1.000000,10


In [162]:
print(f"Média da acurácia: {scores.mean():.4f}")
print(f"Desvio padrão da acurácia: {scores.std():.4f}")

Média da acurácia: 0.9899
Desvio padrão da acurácia: 0.0095


In [163]:
X_new_tfidf = vectorizer.transform(df['DESCRIÇÃO'])

In [164]:
df['Categoria_Prevista'] = rf_model.predict(X_new_tfidf )

In [165]:
df.head(20)

,DATA,DESCRIÇÃO,DOCUMENTO,CRÉDITO,DÉBITO,SALDO,Categoria,Categoria_Codificada,Categoria_Prevista
0,1/4/2022,pg pinternet white martins,10.002.006,NaN,"-R$ 30,90","R$ 16.284,18",Despesas Diversas,2,2
1,1/4/2022,pg pinternet med curitiba,10.001.995,NaN,"-R$ 100,00","R$ 16.184,18",Saúde,8,8
2,1/4/2022,pg pinternet springer carrier,10.002.010,NaN,"-R$ 358,91","R$ 15.825,27",Equipamentos,5,5
3,1/4/2022,credito ted pagseguro internet ltda,33.834.846,"R$ 846,95",NaN,"R$ 16.672,22",Serviço Prestado,10,10
4,1/7/2022,pg pinternet aluguel,10.005.388,NaN,"-R$ 3.046,25","R$ 13.625,97",Aluguel,0,0
5,1/10/2022,prest emprest,8,NaN,"-R$ 1.418,47","R$ 12.207,50",Empréstimos,4,4
6,1/10/2022,db allianz seguro spin de,84624000166,NaN,"-R$ 144,05","R$ 12.063,45",Seguro,9,9
7,1/11/2022,credito ted pagseguro internet ltda,33.655.713,"R$ 1.449,70",NaN,"R$ 13.513,15",Serviço Prestado,10,10
8,1/17/2022,db cotas,19.911,NaN,"-R$ 59,50","R$ 13.453,65",Despesas Diversas,2,2
9,1/17/2022,credito ted pagseguro internet ltda,16,"R$ 1.517,30",NaN,"R$ 14.970,95",Serviço Prestado,10,10


In [ ]:
# listar as categorias em ordem crescente

categorias = df[['Categoria', 'Categoria_Codificada']].drop_duplicates().sort_values('Categoria_Codificada')
categorias_dict = categorias.set_index('Categoria_Codificada')['Categoria'].to_dict()

categorias_dict

{0: 'Aluguel',
 1: 'Contas de Consumo',
 2: 'Despesas Diversas',
 3: 'Educação',
 4: 'Empréstimos',
 5: 'Equipamentos',
 6: 'Impostos e Taxas',
 7: 'Pagamentos e Créditos',
 8: 'Saúde',
 9: 'Seguro',
 10: 'Serviço Prestado',
 11: 'Serviços Digitais',
 12: 'Transporte'}

In [171]:
# Função pra aplicar o modelo recebendo um arquivo csv

categorias_dict

def classificar_despesas(input_path, output_path):
    df = pd.read_csv(input_path, sep=',')
    df['DESCRIÇÃO'] = df['DESCRIÇÃO'].apply(tratar_dados)
    X_new_tfidf = vectorizer.transform(df['DESCRIÇÃO'])
    df['Categoria_Prevista'] = rf_model.predict(X_new_tfidf)

    df['Categoria_Prevista_Nome'] = df['Categoria_Prevista'].map(categorias_dict)
    
    
    

    df.to_csv(output_path, index=False)

In [172]:
classificar_despesas('./despesas_todos_anos.csv', './despesas_todos_anos_classificadas.csv')

In [173]:
show_data = pd.read_csv('./despesas_todos_anos_classificadas.csv', sep=',')
show_data.head(20)

,DATA,DESCRIÇÃO,DOCUMENTO,CRÉDITO,DÉBITO,SALDO,Categoria_Prevista,Categoria_Prevista_Nome
0,1/4/2022,pg p internet white martins,10.002.006,NaN,"-R$ 30,90","R$ 16.284,18",2,Despesas Diversas
1,1/4/2022,pg p internet med curitiba,10.001.995,NaN,"-R$ 100,00","R$ 16.184,18",8,Saúde
2,1/4/2022,pg p internet springer carrier,10.002.010,NaN,"-R$ 358,91","R$ 15.825,27",5,Equipamentos
3,1/4/2022,credito ted pagseguro internet ltda,33.834.846,"R$ 846,95",NaN,"R$ 16.672,22",10,Serviço Prestado
4,1/7/2022,pg p internet aluguel,10.005.388,NaN,"-R$ 3.046,25","R$ 13.625,97",0,Aluguel
5,1/10/2022,prest emprest,8,NaN,"-R$ 1.418,47","R$ 12.207,50",4,Empréstimos
6,1/10/2022,db allianz seguro spin de,84624000166,NaN,"-R$ 144,05","R$ 12.063,45",9,Seguro
7,1/11/2022,credito ted pagseguro internet ltda,33.655.713,"R$ 1.449,70",NaN,"R$ 13.513,15",10,Serviço Prestado
8,1/17/2022,db cotas,19.911,NaN,"-R$ 59,50","R$ 13.453,65",2,Despesas Diversas
9,1/17/2022,credito ted pagseguro internet ltda,16,"R$ 1.517,30",NaN,"R$ 14.970,95",10,Serviço Prestado


In [174]:
import joblib

# Salvar o modelo RandomForest
joblib.dump(rf_model, "rf_model.pkl")

# Salvar o TfidfVectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']